# README

랭그래프의 기본적인 작업 후 대화 내용을 기억하는 챗봇 관련 랭그래프를 구성해보려고 함

In [12]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import ChatOpenAI
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END
# .graph import StateGraph, END

In [2]:
key = open('../../../api_key.txt','r')
api_key = key.read()
# openai.api_key = api_key

base_ = open('../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [72]:
llm = ChatOllama(model = 'gemma3:12b',
                 base_url=base_url)

# 전체 대화 기억
memory = ConversationBufferMemory(return_messages=True)

In [45]:
llm.invoke("오늘 날씨 어때?").content.strip().lower()

'저는 실시간 날씨 정보를 제공할 수 없습니다. 현재 날씨를 확인하시려면 날씨 앱이나 웹사이트를 이용해 주세요. 예를 들어 다음과 같은 곳에서 확인하실 수 있습니다:\n\n*   **네이버 날씨:** [https://weather.naver.com/](https://weather.naver.com/)\n*   **기상청 날씨누리:** [https://www.weather.go.kr/](https://www.weather.go.kr/)\n\n어느 지역의 날씨를 알고 싶으신가요? 지역을 알려주시면 해당 지역의 날씨를 검색하는 방법을 알려드릴 수 있습니다.'

In [49]:
# Node 1: 질문 내용 구분 함수
def classify_question(stats):
    # input_question = {"question": "3 곱하기 7은 얼마야?"}

    question = stats["question"]
    messages = [
                    {"role": "system", 
                        "content": f"""당신은 질문이 어떤 분야의 질문을 하는 분류하는 분류기입니다. 
                        수학 문제면 'math', 역사 문제면 'history', 과학문제면 'science', 아니면 'general'로만 대답하세요.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]
    response = llm.invoke(messages).content.strip().lower()
    stats['type']=response
    return stats

In [73]:
def generate_response(state, category):
    history = memory.chat_memory.messages  # 이전 대화
    current_question = state["question"]
    
    # 현재 질문과 카테고리에 맞는 프롬프트 구성
    prompt = f"사용자가 이렇게 질문했습니다: '{current_question}'\n"
    if category == "math":
        prompt += "→ 질문에 대해서 개념 및 풀이과정을 상세히 설명해주세요."
    elif category == "history":
        prompt += "→ 역사적 사건에 대한 배경 및 순서, 내용에 대해서 상세히 설명해주세요."
    elif category == "science":
        prompt += "→ 과학적인 근거를 이용한 개념 및 관련 정보 대해서 상세히 설명해주세요."
    else:
        prompt += "→ 사용자의 물음에 친절히 답해주세요."
    
    # 모델 호출
    answer = llm.invoke(history + [{"role": "user", "content": prompt}])
    response = answer.content
    
    # 응답을 메모리에 저장
    memory.chat_memory.add_user_message(current_question)
    memory.chat_memory.add_ai_message(response)
    
    state["response"] = response
    return state

In [74]:
# Node Wrappers
math_node = RunnableLambda(lambda s: generate_response(s, "math"))
history_node = RunnableLambda(lambda s: generate_response(s, "history"))
science_node = RunnableLambda(lambda s: generate_response(s, "science"))
general_node = RunnableLambda(lambda s: generate_response(s, "general"))

In [81]:
def decide_next(state):
    return state["type"]

In [82]:
# ✅ LangGraph 구성
state_type = dict
graph = StateGraph(state_schema=state_type)

graph.add_node("classify", RunnableLambda(classify_question))
graph.add_node("math", math_node)
graph.add_node("history", history_node)
graph.add_node("science", science_node)
graph.add_node("general", general_node)


graph.add_edge("math", END)
graph.add_edge("history", END)
graph.add_edge("science", END)
graph.add_edge("general", END)


In [83]:
graph.set_entry_point("classify")

graph.add_conditional_edges(
    "classify",
    path=RunnableLambda(decide_next),
    path_map={
        "math": "math",
        "history": "history",
        "science": "science",
        "general": "general",
    }
)

In [84]:
chat_agent = graph.compile()


In [88]:
chat_agent.invoke({"question": "근의공식이 뭐야?"})

{'question': '근의공식이 뭐야?',
 'type': 'math',
 'response': '## 사용자의 질문 "근의공식이 뭐야?"에 대한 상세 설명 (개념 및 풀이 과정 포함)\n\n사용자의 질문에 답하기 위해 근의공식의 개념과 풀이 과정을 최대한 자세하고 쉽게 설명하겠습니다.\n\n**1. 왜 근의공식을 알아야 할까요? (도입)**\n\n우선, 근의공식이 왜 중요한지, 왜 배워야 하는지부터 알아봅시다.\n\n* **2차 방정식 풀이:** 우리 주변에는 다양한 문제가 2차 방정식으로 표현될 수 있습니다. 예를 들어, 물체의 운동 궤적, 최적화 문제 등입니다. 하지만 2차 방정식을 인수분해하기 어려울 때가 많습니다. 이때 근의공식이 빛을 발합니다.\n* **수학적 사고력 향상:** 근의공식은 방정식의 해를 찾는 과정을 이해하고, 추상적인 수학적 사고력을 키우는 데 도움이 됩니다.\n* **다른 분야와의 연계:** 물리학, 공학 등 다양한 분야에서 2차 방정식이 사용되므로, 근의공식은 실생활 문제 해결에도 활용될 수 있습니다.\n\n**2. 2차 방정식이란 무엇인가? (개념 정리)**\n\n근의공식을 이해하기 위해서는 2차 방정식에 대한 기본적인 이해가 필요합니다. 2차 방정식은 다음과 같은 형태를 띠는 방정식입니다.\n\n**ax² + bx + c = 0** (단, a ≠ 0)\n\n* **a, b, c:** 상수 (정수, 분수, 실수 등)\n* **x:** 미지수 (우리가 구해야 할 값)\n\n예를 들어: 2x² + 3x - 1 = 0  (a = 2, b = 3, c = -1)\n\n**3. 근의공식이란 무엇인가? (핵심 개념)**\n\n근의공식은 2차 방정식 ax² + bx + c = 0 의 해(근)를 구하는 공식입니다.  즉, x에 어떤 값을 대입하면 방정식이 성립하는 x 값을 찾는 것입니다.\n\n**근의공식:**\n\n**x₁ = (-b + √(b² - 4ac)) / 2a**\n\n**x₂ = (-b - √(b² - 4ac)) / 2a**\

In [ ]:
result = chat_agent.invoke({"question": "세종대왕의 업적은?"})
print(result['question'])
print(result['response'])

In [94]:
print(result['question'])
print(result['response'])

세종대왕의 업적은?
## 세종대왕의 업적 상세 설명: 역사적 배경과 순서, 내용

세종대왕(세종, 1397년 ~ 1450년)은 조선 제4대 왕으로, 한국 역사상 가장 위대한 왕 중 한 명으로 꼽힙니다. 그의 업적은 단순히 훈민정음 창제를 넘어, 과학 기술 발전, 국방 강화, 민생 안정 등 다방면에서 빛을 발했습니다. 세종대왕의 업적을 역사적 배경과 순서, 내용을 자세히 설명하겠습니다.

**1. 역사적 배경: 혼란과 새로운 시대의 요구**

세종대왕이 즉위할 당시 조선은 여러 가지 어려움에 직면했습니다. 이러한 어려움들은 세종대왕이 백성을 위한 정치를 펼치고, 국가의 위상을 높이기 위한 노력의 배경이 되었습니다.

* **정치적 혼란:** 성리학의 영향력이 강해지면서 사림 세력과 훈구 세력 간의 갈등이 심화되었습니다. 이는 왕권 약화로 이어져 정책 추진에 어려움을 겪게 했습니다.
* **사회적 불평등:** 양반과 평민 간의 경제적, 사회적 격차가 심화되어 민생이 어려웠습니다. 특히 한자를 읽고 쓰지 못하는 백성들의 불만이 컸습니다.
* **문맹률의 심각성:** 한자의 복잡성으로 인해 일반 백성은 한자를 읽고 쓸 수 없어, 정치 참여와 정보 습득에 어려움을 겪었습니다.
* **잦은 외침:** 왜구의 침략이 끊이지 않아 백성들의 삶과 안전을 위협했습니다.
* **농업 생산의 어려움:** 흉년이 잦아 농민들의 생활이 어려웠고, 이는 사회 불안으로 이어졌습니다.

**2. 즉위와 초기 노력 (1418-1420): 훈정치 기틀 마련**

* **1418년 즉위:** 세종은 태종의 셋째 아들로, 훈구 세력의 지지 속에서 즉위합니다.
* **1418년 잦은 간신들의 폐출:** 간신들의 폐출을 통해 왕권을 강화하고, 공정한 관료 임용을 시작합니다.
* **1419년 신미대반란 진압:** 신미대반란을 진압하며 왕권의 안정을 확립합니다.

**3. 훈민정음 창제와 민본주의 정책 (1420-1430): 백성을 위한 개혁**

* **1420년 훈민정음 창제:** 백성들이 쉽게 글

In [90]:
chat_agent.invoke({"question": "일식과 월식은 뭐야?"})


{'question': '일식과 월식은 뭐야?',
 'type': 'science',
 'response': '## 일식과 월식: 천체 현상의 정석, 상세 과학 해설\n\n"일식과 월식은 뭐야?" 라는 질문에 대한 답변입니다. 이 두 현상은 하늘에서 관측되는 아름다운 천체 현상이지만, 그 원리는 꽤 흥미롭습니다. 과학적인 근거를 바탕으로 일식과 월식의 개념, 원리, 그리고 관련 정보를 상세히 설명하겠습니다.\n\n**1. 일식(Solar Eclipse): 태양이 가려지는 순간**\n\n* **개념:** 일식은 달이 태양과 지구 사이에 위치하여 태양을 가리는 현상입니다. 달의 그림자가 지구 표면에 드리워지면서 발생합니다.\n* **원리:** 지구, 달, 태양이 정확히 일직선상에 놓여야 일식이 발생합니다. 달의 궤도가 지구 궤도와 약 5도 기울어져 있기 때문에 일식이 자주 발생하지 않습니다.\n* **종류:**\n    * **개기일식 (Total Solar Eclipse):** 달이 태양을 완전히 가려 태양이 보이지 않는 현상입니다. 일식 전체가 관측 가능하며, 태양 코로나(태양 대기의 가장 바깥층)를 관측할 수 있는 귀한 기회입니다.\n    * **부분일식 (Partial Solar Eclipse):** 달이 태양의 일부만 가리는 현상입니다.\n    * **반쪽그림자일식 (Annular Solar Eclipse):** 달이 태양을 완전히 가리지 못하고 태양의 가장자리가 보이는 현상입니다. 지구에서 달과의 거리가 멀 때 발생합니다.\n* **관측 시 주의사항:** **반드시 태양 필터를 사용해야 합니다!** 맨눈으로 태양을 보면 심각한 시력 손상을 입을 수 있습니다.\n\n**2. 월식(Lunar Eclipse): 달이 붉게 물드는 밤**\n\n* **개념:** 월식은 지구가 태양과 달 사이에 위치하여 달에 그림자를 드리우는 현상입니다. 달이 지구의 그림자 속으로 들어가면서 발생합니다.\n* **원리:** 지구, 달, 태양이 일직선상에 놓여야 월식이 발생합니

In [91]:
chat_agent.invoke({"question": "저녁 뭐 먹을까?"})

{'question': '저녁 뭐 먹을까?',
 'type': 'general',
 'response': '저녁 메뉴 고민이시군요! 어떤 음식을 좋아하시나요? \n\n몇 가지 추천을 드리자면…\n\n* **간단한 메뉴:** 김치볶음밥, 라면, 계란찜 같은 간단한 메뉴도 좋구요.\n* **든든한 메뉴:** 삼겹살, 파스타, 찜닭처럼 푸짐한 메뉴도 괜찮죠!\n* **특별한 메뉴:** 평소에 잘 안 먹는 음식이나 새로운 레시피에 도전해 보는 것도 재밌을 거예요.\n\n혹시 특별히 생각나는 메뉴가 있으신가요? 아니면 냉장고에 있는 재료를 알려주시면, 그걸로 만들 수 있는 요리를 추천해 드릴 수도 있어요! 😊'}